# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.



In [0]:
!pip install syft
!pip install numpy

     |████████████████████████████████| 215kB 2.9MB/s 
     |████████████████████████████████| 204kB 40.8MB/s 
     |████████████████████████████████| 450kB 44.7MB/s 
     |████████████████████████████████| 71kB 23.7MB/s 
     |████████████████████████████████| 389kB 49.9MB/s 
     |████████████████████████████████| 256kB 45.6MB/s 
     |████████████████████████████████| 1.4MB 43.1MB/s 
     |████████████████████████████████| 51kB 20.5MB/s 
     |████████████████████████████████| 276kB 49.9MB/s 
     |████████████████████████████████| 122kB 47.6MB/s 
  Stored in directory: /root/.cache/pip/wheels/ad/9a/f4/3105b5209674ac77fcca7fede95184c62a95df0196888e0e76
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
Successfully built zstd pyyaml
  Found existing installation: msgpack 0.5.6
    Uninstalling msgpack-0.5.6:
      Successfully uninstalled msgpack-0.5.6
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
   

In [0]:
import torch as th

In [0]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [0]:
y = x + x

In [0]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [0]:
import syft as sy

W0625 18:28:46.406823 140348963522432 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0-rc1.so'
W0625 18:28:46.426077 140348963522432 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
hook = sy.TorchHook(th)  # this line of code create an instance of th with its backend APIs modified with Syft functions

In [0]:
th.tensor([1,2,3,4,5]) # notice that Torch functionalities still behave the same

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [0]:
bob = sy.VirtualWorker(hook, id="bob") # creates a virtual worker (a simulaion to interface to Bob machines)

In [0]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5])

In [0]:
x_ptr =  x.send(bob) # send this data to bob

In [0]:
bob._objects

{66334714076: tensor([1, 2, 3, 4, 5])}

In [0]:
x_ptr

(Wrapper)>[PointerTensor | me:48348495859 -> bob:66334714076]

In [0]:
x.location

<VirtualWorker id:bob #objects:1>

In [0]:
x.id_at_location

66334714076

In [0]:
x.id

48348495859

In [0]:
x.owner

<VirtualWorker id:me #objects:0>

In [0]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [0]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [0]:
bob._objects

{}

# Project: Experience with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [0]:
# create a second Virtual worker and call it Alice


In [0]:
# 1 - create some data (a tensor)
# 2- send the data to bob and alice


In [0]:
# notice what does the send fucntion on two workers return; a multi-pointer
x_ptr

In [0]:
# what does .child on the pointer object return?
# what does .child.child on the pointer object return? 

In [0]:
# try the .get() on the pointer


In [0]:
# try the .get(sum_results=True) on your pointer


# Lesson: Introducing Remote Arithmetic

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [0]:
x # a pointer to a remote tensor located at bob

(Wrapper)>[PointerTensor | me:96502616908 -> bob:83999397113]

In [0]:
y # a pointer to another remote tensor located at bob

(Wrapper)>[PointerTensor | me:35900922850 -> bob:59092289169]

In [0]:
z = x + y # treat those tensors (i.e., x and y) as local tensors, but they are actually executed remotely
z

(Wrapper)>[PointerTensor | me:12996980457 -> bob:54222168809]

In [0]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [0]:
z = th.add(x,y) # we can also use Torch operation on these tensors to be executed remotely
z

(Wrapper)>[PointerTensor | me:28437210120 -> bob:28437210120]

In [0]:
z = z.get() # yup! same results!
z

tensor([2, 3, 4, 5, 6])

In [0]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [0]:
z = (x + y).sum()
z = z.get()
z

tensor(20., requires_grad=True)

In [0]:
z.backward() 

tensor(20., requires_grad=True)

In [0]:
x = x.get()

In [0]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [0]:
x.grad

tensor([1., 1., 1., 1., 1.])

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). 

Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [0]:
import torch as th
import syft as sy

hook = sy.TorchHook(th)  
bob = sy.VirtualWorker(hook, id="bob")

bob

W0702 17:55:24.924933 140275637208960 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0702 17:55:24.946931 140275637208960 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



<VirtualWorker id:bob #objects:0>

In [0]:
# create some toy data for our model
input_data = th.tensor([[1.,1],[0.5,1],[1,0],[0,0]], requires_grad=True).send(bob)
output_data = th.tensor([[1.],[1],[0],[0]], requires_grad=True).send(bob)

In [0]:
# send the data to bob

torch.Size([2])

In [0]:
# create some linear weights and send them to bob
weights = th.tensor([[0.01],[0.01]], requires_grad = True).send(bob)

In [0]:
# create a linear model and train it on Bob's machine
# remember how to create a linear model? No? :( 
# Here's the main steps:
#    1- find a prediction
#    2- calcualte the loss (a mean square loss)
#    3- backpropogate using the backword() function
#    4- DO NOT forget to clear your gradients after updating the weights
#    weights.data.sub_(weights.grad * lr)

prediction = input_data.mm(weights)
prediction

(Wrapper)>[PointerTensor | me:15127782505 -> bob:52047116384]

In [0]:
lr = 0.1

for i in range(10):
  prediction = input_data.mm(weights)
  loss = ((prediction - output_data)**2).mean()
  loss.backward()
  weights.data.sum_(weights.grad * lr)
  weights.grad *= 0

  print(loss.get().data)

tensor(0.0461)
tensor(0.0420)
tensor(0.0386)
tensor(0.0357)
tensor(0.0332)
tensor(0.0309)
tensor(0.0288)
tensor(0.0269)
tensor(0.0252)
tensor(0.0236)


# Lesson: Garbage Collection and Common Errors


In [0]:
bob = bob.clear_objects() # clear the contents of a remote object

In [0]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
bob._objects

{85910937763: tensor([1, 2, 3, 4, 5])}

In [0]:
del x  # delete the pointer to the remote object

In [0]:
bob._objects

{}

In [0]:
x.child.garbage_collect_data  # True by default

True

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
bob._objects

{84615314564: tensor([1, 2, 3, 4, 5])}

In [0]:
x = "asdf"

In [0]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
x

(Wrapper)>[PointerTensor | me:39795892651 -> bob:12911623213]

In [0]:
bob._objects

{12911623213: tensor([1, 2, 3, 4, 5])}

In [0]:
x = "asdf"

In [0]:
bob._objects  # some error from Jupyter is casuing this to appear!

{12911623213: tensor([1, 2, 3, 4, 5])}

In [0]:
del x

In [0]:
bob._objects

{12911623213: tensor([1, 2, 3, 4, 5])}

In [0]:
bob = bob.clear_objects() # erase force 
bob._objects

{}

In [0]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
bob._objects # notice that there is only a single tensor in bob

{15601760445: tensor([1, 2, 3, 4, 5])}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [0]:
z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually locatedon another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(bob) on the other tensor.

Tensor A: [PointerTensor | me:46419059800 -> bob:14412738960]
Tensor B: tensor([1, 1, 1, 1, 1])

In [0]:
alice = sy.VirtualWorker(hook, id="alice")
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [0]:
z = x + y

TensorsNotCollocatedException: ignored

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [34]:
from torch import nn, optim

In [35]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [36]:
# A Toy Model
model = nn.Linear(2,1)

In [37]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [38]:
def train(iterations=10):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(3.8457)
tensor(1.1061)
tensor(0.6067)
tensor(0.3813)
tensor(0.2440)
tensor(0.1565)
tensor(0.1005)
tensor(0.0646)
tensor(0.0415)
tensor(0.0267)


In [8]:
# nothing was federated up to this point!

import torch as th
import syft as sy

hook = sy.TorchHook(th)  
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")



In [9]:
# create local datasets at Bob and Alice
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [10]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]
datasets

[((Wrapper)>[PointerTensor | me:90462569521 -> bob:95065373770],
  (Wrapper)>[PointerTensor | me:41520456612 -> bob:79594524149]),
 ((Wrapper)>[PointerTensor | me:52509294717 -> alice:51892992123],
  (Wrapper)>[PointerTensor | me:65661429222 -> alice:62718113000])]

In [13]:
def train(iterations=1):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do training on the remote machine
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [14]:
train()

RuntimeError: Expected object of scalar type Long but got scalar type Float for argument #2 'mat2'

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [0]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
x = x.send(alice)

In [0]:
bob._objects

{681755396: tensor([1, 2, 3, 4, 5])}

In [0]:
alice._objects

{59232059007: (Wrapper)>[PointerTensor | alice:59232059007 -> bob:681755396]}

In [0]:
y = x + x

In [0]:
y

(Wrapper)>[PointerTensor | me:35076229352 -> alice:14700048988]

In [0]:
bob._objects

{681755396: tensor([1, 2, 3, 4, 5]), 61259427323: tensor([ 2,  4,  6,  8, 10])}

In [0]:
alice._objects

{14700048988: (Wrapper)>[PointerTensor | alice:14700048988 -> bob:61259427323],
 59232059007: (Wrapper)>[PointerTensor | alice:59232059007 -> bob:681755396]}

In [0]:
jon = sy.VirtualWorker(hook, id="jon")

In [0]:
bob.clear_objects()
alice.clear_objects()



<VirtualWorker id:alice #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)
y = th.tensor([1,2,3,4,5]).send(bob).send(jon)

In [0]:
z = x + y

TensorsNotCollocatedException: ignored

In [0]:
x = x.get() # to get the data back
x

(Wrapper)>[PointerTensor | me:59854104348 -> bob:36602545298]

In [0]:
bob._objects

{36602545298: tensor([1, 2, 3, 4, 5]), 89784608003: tensor([1, 2, 3, 4, 5])}

In [0]:
alice._objects

{}

In [0]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [0]:
bob._objects

# Lesson: Pointer Chain Operations

In [0]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
bob._objects

{36597101211: tensor([1, 2, 3, 4, 5])}

In [0]:
alice._objects

{}

In [0]:
x.move(alice)

(Wrapper)>[PointerTensor | me:16851411436 -> alice:16851411436]

In [0]:
x

(Wrapper)>[PointerTensor | me:16851411436 -> alice:16851411436]

In [0]:
bob._objects

{}

In [0]:
alice._objects

{16851411436: tensor([1, 2, 3, 4, 5])}

In [0]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [0]:
bob._objects

{81869613150: tensor([1, 2, 3, 4, 5])}

In [0]:
alice._objects

{99685627431: (Wrapper)>[PointerTensor | alice:99685627431 -> bob:81869613150]}

In [0]:
x

(Wrapper)>[PointerTensor | me:83688287202 -> alice:99685627431]

In [0]:
x.remote_get() # inplace operation

(Wrapper)>[PointerTensor | me:83688287202 -> alice:99685627431]

In [0]:
bob._objects

{}

In [0]:
alice._objects

{99685627431: tensor([1, 2, 3, 4, 5])}

In [0]:
x.move(bob)

(Wrapper)>[PointerTensor | me:83688287202 -> bob:83688287202]

In [0]:
x

(Wrapper)>[PointerTensor | me:83688287202 -> bob:83688287202]

In [0]:
bob._objects

{83688287202: tensor([1, 2, 3, 4, 5])}

In [0]:
alice._objects

{}

**Exercise 1:**

- Create a toy dataset
- Create a toy linear model
- Train the dataset using Federated Learning, but do not share the gradiant with the main server, instead use the move function to send the gradients to one of the participating workers, aggregate the model, then send the final model to the main server (i.e., local worker).



---


**Exercise 2:**

While exercise 1 illustrates a good example of preseving privacy in FL, we still had to expose the gradients to one of the participating workers. To avoid this limitation, you need to send the gradiants to a TRUSTED thirdparty (trusted aggregator) who will aggregate the models and then send the final model to the server (local worker). In this way, we assure that none of the participating workers can access the aggregated model!

1. create a dataset for each worker (create two)
2. create a model for each worker and train it remotely on each worker
3. send those two models using the *move()* function to a third worker
4. the third workers aggregates the two models (find their mean)
5. send the aggregated model to the main server (local worker) using the *get()* function



---

To set the weights: `model.weight.set_()`



In [1]:
# Exercise 1
import syft as sy
import torch as th
from torch import nn, optim

In [2]:
# create workers

hook = sy.TorchHook(th) 
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secureWorker = sy.VirtualWorker(hook, id="secureWorker")

In [3]:
# Make each worker aware of the other workers
bob.add_workers([alice,secureWorker])
alice.add_workers([bob,secureWorker])
secureWorker.add_workers([bob, alice])
print(bob._objects)
print(alice._objects)
print(secureWorker._objects)

{}
{}
{}


In [4]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [5]:
# create local datasets at Bob and Alice
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

datasets = [(data_bob, target_bob), (data_alice, target_alice)]
print(bob._objects)
print(alice._objects)
print(secureWorker._objects)

datasets

{18027065044: tensor([[1., 1.],
        [0., 1.]], requires_grad=True), 84641949897: tensor([[1.],
        [1.]], requires_grad=True)}
{29159554854: tensor([[1., 0.],
        [0., 0.]], requires_grad=True), 31766297094: tensor([[0.],
        [0.]], requires_grad=True)}
{}


[((Wrapper)>[PointerTensor | me:53279077988 -> bob:18027065044],
  (Wrapper)>[PointerTensor | me:77593601157 -> bob:84641949897]),
 ((Wrapper)>[PointerTensor | me:45046371518 -> alice:29159554854],
  (Wrapper)>[PointerTensor | me:99405523599 -> alice:31766297094])]

In [6]:
# create a linear model at local worker
model = nn.Linear(2,1)

In [7]:
# Send copies of the linear model to alice and bob
bob_mod = model.copy().send(bob)
alice_mod = model.copy().send(alice)

In [8]:
print(bob_mod)

opt = optim.SGD(params=bob_mod.parameters(), lr=0.1)
opt.zero_grad()
pred = bob_mod(data_bob)
loss = ((pred - target_bob)**2).sum()
loss.backward()
opt.step()
print(loss.get())

print(alice_mod)



opt = optim.SGD(params=alice_mod.parameters(), lr=0.1)
opt.zero_grad()
pred = alice_mod(data_alice)
loss = ((pred - target_alice)**2).sum()
loss.backward()
opt.step()
print(loss.get())



Linear(in_features=2, out_features=1, bias=True)
tensor(8.9852, requires_grad=True)
Linear(in_features=2, out_features=1, bias=True)
tensor(0.6257, requires_grad=True)


In [9]:
# move the models to the third worker
bob_mod.move(secureWorker)
alice_mod.move(secureWorker)
print(secureWorker._objects)
# aggreegate the models
aggie = (bob_mod.weight + alice_mod.weight) / 2
# send the model back to the local worker
aggie = aggie.get()
print(aggie.type())
#aggie
##aggie = aggie.data
model.weight.requires_grad = False
model.weight.set_(aggie)
print(model)


{80754467267: Parameter containing:
tensor([[0.6526, 0.2752]], requires_grad=True), 54869474049: Parameter containing:
tensor([0.1742], requires_grad=True), 36847119405: Parameter containing:
tensor([[ 0.3384, -0.5711]], requires_grad=True), 33742874181: Parameter containing:
tensor([-0.4542], requires_grad=True)}
torch.FloatTensor
Linear(in_features=2, out_features=1, bias=True)


In [ ]:
# train the models at alice and bob

def train(iterations=1):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do training on the remote machine
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()
            
            # Don't get smarter model back
            ##model = model.get()

            print(loss.get())
        
train()

In [11]:

# move the models to the third worker
bob_mod.move(secureWorker)
alice_mod.move(secureWorker)
# aggreegate the models
aggie = (bob_mod.sum() + alice_mod.sum()) / 2
# send the model back to the local worker
aggie = aggie.get()
aggie

AttributeError: 'Parameter' object has no attribute 'child'

In [0]:
def train(iterations=1):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do training on the remote machine
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())